In [1]:
import pandas as pd
from subprocess import check_call
import json
import glob

Código baseado nesse Gist: https://gist.github.com/gabrielzanlorenssi/9a93c16ed2ca73c782c15a11cfd00591

In [19]:
!wget -q https://resultados.tse.jus.br/oficial/ele2022/545/config/mun-e000545-cm.json

In [15]:
with open('mun-e000545-cm.json') as _f:
    _json = json.load(_f)
    
# abrir arquivo municipal para obter os links de download
dfs = []
for row in _json['abr']:
    state = row['cd']
    for mu in row['mu']:
        dfs.append({
            'state' : state,
            'id_ibge' : mu['cdi'],
            'id_tse' : mu['cd']
        })
munis = pd.DataFrame(dfs)
!mkdir -p json/2t/tse/

# baixar cada arquivo municipal
for mu in dfs:
    url_muni = "".join([str(x) for x in ["https://resultados.tse.jus.br/oficial/ele2022/",
                  545 ,# codigo do turno
                  "/dados/", # dados consolidados
                  mu['state'].lower(), # indentificacao do estado
                  "/", mu['state'].lower(), # identificacao do estado
                  mu['id_tse'], # codigo do tse do municipio
                  "-c0001", # cargo
                  "-e000545", # pleito
                  '-v', # tipo de arquivo
                  ".json"]])
    if len(glob.glob(f'json/2t/tse/{mu["id_ibge"]}.json')) == 0:
        check_call(['wget',url_muni,'-q','-O',f'json/2t/tse/{mu["id_ibge"]}.json'], timeout=20)

In [16]:
# gerar o arquivo CSV
_erros = 0
dfs = []
for a in glob.glob('json/2t/tse/*.json'):
    with open(a) as _f:
        #print(a)
        try:
            _json = json.load(_f)
        except:
            print('erro', a)
            _erros += 1
            continue
        for row in _json['abr']:
            if row['tpabr'] == "MU":
                _df = pd.DataFrame(row['cand'])
                _df['id_ibge'] = a
                _df['st'] = row['st']
                _df['pst'] = row['pst']
                dfs.append(_df)
# MANUALMENTE: Em caso de erro, apague o respectivo arquivo e rode o código da célula anterior novamente até que não haja mais nenhum erro!

In [17]:
assert _erros == 0

In [18]:
# gerar o arquivo municipal completo
df = pd.concat(dfs)
df['id_ibge'] = df['id_ibge'].map(lambda x: x.split('/')[-1].replace('.json',''))
df['pst'] = df['pst'].str.replace(',','.')
df['pvap'] = df['pvap'].str.replace(',','.')
del df['e']
del df['seq']
df['n'] = df['n'].map({
    '13' : 'LULA',
    '22' : 'JAIR BOLSONARO'
})
df = df.rename(columns={
    'n' : 'nc',
    'vap' : 'votos',
    'st' : 'secoes_totais',
    'pst' : 'urnas_apuradas',
    'pvap' : 'prop'
})
df = df[['id_ibge','nc','votos','urnas_apuradas','prop','secoes_totais']].sort_values(by=['id_ibge','nc']).reset_index(drop=True)
df.to_csv('municipios_2t.csv',index=False)
df

,id_ibge,nc,votos,urnas_apuradas,prop,secoes_totais
0,1100015,JAIR BOLSONARO,9768,100.00,71.16,72
1,1100015,LULA,3959,100.00,28.84,72
2,1100023,JAIR BOLSONARO,39901,100.00,79.15,261
3,1100023,LULA,10513,100.00,20.85,261
4,1100031,JAIR BOLSONARO,2405,100.00,68.50,14
...,...,...,...,...,...,...
11135,5222203,LULA,1721,100.00,64.80,12
11136,5222302,JAIR BOLSONARO,1959,100.00,51.02,18
11137,5222302,LULA,1881,100.00,48.98,18
11138,5300108,JAIR BOLSONARO,1041331,100.00,58.81,6748
